# Deploy Image Classification with OpenVINO Model Server in OpenShift 

We will show you how to deploy OpenVINO Model Server (OVMS) in an OpenShift cluster and how to run a gRPC prediction request to the AI inference service.

Requirements:
- OpenShift cluster with the API access to a project
- installed [OpenVINO Model Server Operator](https://catalog.redhat.com/software/operators/search?q=openvino)
- JupyterLab environment with Python3 deployed in the cluster

If you don't have an OpenShift account, you can sign up for 30 or 60 day [free trial of Red Hat OpenShift](https://www.openshift.com/try).

## Login to OpenShift with API Token

First, let's login to OpenShift cluster using `oc` tool. 

In the Red Hat OpenShift console, click on your username and select `Copy login command`.

![copy-login.png](notebook-files/copy-login.png)

Click on `Display Token` and your API token will appear.

![log-in-with-token.png](notebook-files/log-in-with-token.png)

Copy `Log in with token` command and paste it in the cell below. The command has your `<user-API-token>` and `<cluster-DNS-name>`.

In [ ]:
!oc login --token=<user-API-token> --server=https://api.<cluster-DNS-name>:6443

Create `ovms` project and go to this project.

In [ ]:
!oc new-project ovms
!oc project ovms

## Create MinIO Storage

OpenVINO Model Server exposes DL models over gRPC and REST interface. The models can be stored in cloud storage like AWS S3, Google Storage or Azure Blobs. In OpenShift and Kubernetes, Persistent Storage Claim could be used as well. In this tutorial, we will use MinIO service which is an equivalent of AWS S3.

Let's create a MinIO service.

In [ ]:
!oc apply -f minio.yaml

Next step is to download `mc`, MinIO Client.

In [ ]:
!wget https://dl.min.io/client/mc/release/linux-amd64/mc

Change the access permissions on `mc`, so we can run commands with it.

In [ ]:
!chmod 755 mc

Let's make an alias for the MinIO service.

In [ ]:
!./mc alias set minio http://minio-service.ovms:9000 minio minio123

Create a `minio/models` bucket; it's where we will store our model.

In [ ]:
!./mc mb minio/models

## Create ResNet Model Repository

Now, we will upload the models for serving in the OpenVINO Model Server. We will use [ResNet50 model in ONNX format](https://github.com/onnx/models/tree/master/vision/classification/resnet).


Copy the ResNet model from its repository.

In [ ]:
!curl -L --create-dir https://github.com/onnx/models/raw/master/vision/classification/resnet/model/resnet50-caffe2-v1-9.onnx -o resnet/1/resnet50-caffe2-v1-9.onnx

Now, copy the ResNet model into MinIO bucket.

In [ ]:
!./mc cp --recursive resnet minio/models/

Let's make sure the model has been successfully copied.

In [ ]:
!./mc ls -r minio/models/resnet

## Deploy OpenVINO Model Server

Let's deploy an OpenVINO Model Server service in the cluster. We will create a serving of a single model, ResNet50 model in ONNX format, which we uploaded into MinIO bucket.

Here's the yaml file used to configure the OVMS service. We specified name to be `ovms-resnet` and `model_path` to be `s3://minio-service:9000/models/resnet`. Also, we defined `model_name` here.

In [ ]:
!cat ovms-resnet.yaml

Run the cell below to create new OVMS service called `ovms-resnet`.

In [ ]:
!oc apply -f ovms-resnet.yaml

Let's see if pod and service were created. They should start with `ovms-resnet`.

In [ ]:
!oc get pod
!oc get service

Let's check if the OpenVINO Model Server service is running by making an API request via cURL.

In [ ]:
!curl http://ovms-resnet.ovms.svc:8081/v1/models/resnet

## Run a Prediction Request


We will run image classification on this image by making gRPC API requests to the `ovms-resnet` OVMS service.

![image](bee.jpeg)

Import Python packages needed to run prediction requests.

In [ ]:
import grpc
import numpy as np
import classes
from tensorflow import make_tensor_proto, make_ndarray, make_tensor_proto
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import cv2

Next, we will create two functions to make a NumPy array from input image. The array will be transformed to required format and data range.

In [ ]:
def preprocess(img_data):
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
         # for each pixel in each channel, divide the value by 255 to get value between [0, 1] and then normalize
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data

def getJpeg(path, size):
    with open(path, mode='rb') as file:
        content = file.read()

    img = np.frombuffer(content, dtype=np.uint8)
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)  # BGR format
    # format of data is HWC
    # add image preprocessing if needed by the model
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32')
    #convert to NHWC
    img = img.transpose(2,0,1)
    # normalize to adjust to model training dataset
    img = preprocess(img)
    img = img.reshape(1,3,size,size)
    print(path, img.shape, "; data range:",np.amin(img),":",np.amax(img))
    return img

Let's create a NumPy array from the bee image. Then, we will submit a gRPC request to the `ovms-resnet` service and print out the result.

In [ ]:
img1 = getJpeg('bee.jpeg', 224)

channel = grpc.insecure_channel("ovms-resnet.ovms.svc:8080")
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = "resnet"
request.inputs["gpu_0/data_0"].CopyFrom(make_tensor_proto(img1, shape=(img1.shape)))
result = stub.Predict(request, 10.0) # result includes a dictionary with all model outputs

output = make_ndarray(result.outputs["gpu_0/softmax_1"])
ma = np.argmax(output)
print("Class with highest score: {}".format(ma))
print("Detected class name: {}".format(classes.imagenet_classes[ma]))

## Run a Prediction Request on Your Image

You can try it with your own image. In the first line of the next cell, change `<path-to-image>` to the path of the image on which you would like to run classification inference.

In [ ]:
img1 = getJpeg('<path-to-image>', 224)

channel = grpc.insecure_channel("ovms-resnet.ovms.svc:8080")
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = "resnet"
request.inputs["gpu_0/data_0"].CopyFrom(make_tensor_proto(img1, shape=(img1.shape)))
result = stub.Predict(request, 10.0) # result includes a dictionary with all model outputs

output = make_ndarray(result.outputs["gpu_0/softmax_1"])
ma = np.argmax(output)
print("Class with highest score: {}".format(ma))
print("Detected class name: {}".format(classes.imagenet_classes[ma]))

## Cleanup

Let's free up resources.

In [ ]:
!oc delete ovms ovms-resnet
!oc delete deploy minio
!oc delete service minio-service

## Next Steps

In this notebook, you have learned how to deploy an OVMS service with ResNet50 image classification model in an OpenShift cluster. Next, you can explore other OpenShift OVMS notebooks:

- [Deploy Image Classification with OpenVINO Model Server in OpenShift](../401-model-serving-openshift-resnet/ovms-openshift-resnet.ipynb)
- [Face Detection Multi Model OpenVINO Model Server Deployment in OpenShift](../403-model-serving-openshift-face-detection-dag/ovms-openshift-face-detection-dag.ipynb)